# **Class 9: Part 2 - LangChain Introduction**

1. **PromptTemplates**: We will start with PromptTemplates, which are structures that allow more dynamic and reusable prompts for language models. By using placeholders, you will be able to inject custom data into your prompts, making them more adaptable for various applications across different contexts.

2. **Output Parsers**: Next, we will discuss Output Parsers, tools that help interpret and process the raw outputs from language models. These parsers can transform responses into structured formats like JSON or extract specific information, making it easier to handle and utilize the output in your applications.

3. **Memory**: Memory mechanisms in LangChain will allow models to retain context across interactions. This is crucial for maintaining coherent dialogues, especially when building conversational applications. You will learn how different types of memory can be implemented to store and retrieve past interactions.

4. **Chains**: We will explore Chains, which are sequences of calls to language models and other utilities. Chains will enable you to link multiple operations together, allowing more complex interactions and processing workflows while maintaining modularity.

5. **Creating a Chatbot**: Finally, we will apply all these concepts to create a simple chatbot. By using prompt templates for questions, output parsers for understanding responses, chains for coordinating the conversation flow, and memory for context retention, you will build a basic yet functional conversational agent.

# LangChain
LangChain is an innovative framework designed to streamline the development of applications powered by large language models (LLMs). It offers a set of tools and abstractions that make it easier to build complex, functionality-rich applications by orchestrating interactions with these models. At its core, LangChain provides components such as PromptTemplates, Output Parsers, and Chains, which facilitate the creation of dynamic prompts, structured output processing, and sequences of operations, respectively.


In [366]:
from openai import OpenAI
client = OpenAI()

def get_completion(prompt, model='gpt-3.5-turbo', **kwargs):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        **kwargs,# this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:
from langchain.chat_models import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')
llm

## Main Features

### PromptTemplates

#### Motivation

Imagine we want to translate an email.

In [ ]:
email = """
Exmo(a) Sr(a), \
Espero que este email o(a) encontre bem. \
Venho por este meio solicitar informações detalhadas sobre os serviços que a vossa empresa oferece. \
Estou interessado(a) em saber mais sobre os vossos produtos e soluções, bem como os preços e condições de pagamento. \
Gostaria também de agendar uma reunião para discutir possíveis parcerias e oportunidades de negócio. \
Por favor, indiquem-me a disponibilidade da vossa equipa para um encontro presencial ou virtual. \
Agradeço desde já a vossa atenção e aguardo ansiosamente pela vossa resposta. \
Com os melhores cumprimentos"""

In [ ]:
style = """American English \
in a calm and respectful tone
"""

We prepare a prompt that combines the email and the target language.

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{email}```
"""

Using the LangChain OpenAI Client returns an object of type AIMessage.

In [ ]:
response = llm.invoke(prompt)
response

AIMessage(content='Dear Sir/Madam,\n\nI hope this email finds you well. I am reaching out to request detailed information about the services your company offers. I am interested in learning more about your products and solutions, as well as pricing and payment terms. I would also like to schedule a meeting to discuss potential partnerships and business opportunities. Please let me know the availability of your team for an in-person or virtual meeting. Thank you for your attention and I look forward to your response.\n\nBest regards,', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 190, 'total_tokens': 287, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7010db80-5136-4dc

In [ ]:
print(response.content)

Dear Sir/Madam,

I hope this email finds you well. I am reaching out to request detailed information about the services your company offers. I am interested in learning more about your products and solutions, as well as pricing and payment terms. I would also like to schedule a meeting to discuss potential partnerships and business opportunities. Please let me know the availability of your team for an in-person or virtual meeting. Thank you for your attention and I look forward to your response.

Best regards,


What if we want to repeate the process for another language?

#### Solution

It is useful to reuse good and complex prompts and detailed.
Prompt Templates are a good abstraction.

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [ ]:
# Define the system and human message templates
system_message_template = SystemMessagePromptTemplate.from_template(
    """Translate the text that is delimited by triple backticks into a style that is {style}."""
)

human_message_template = HumanMessagePromptTemplate.from_template(
    "text: ```{text}```"
)

# Combine them into a chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    system_message_template,
    human_message_template,
])

In [ ]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.')

In [ ]:
prompt_template.messages[0].prompt.input_variables

['style']

In [ ]:
# Define the style and lyrics
style = """English UK very polite and respectful as if you were royalty, if necessary corrects the grammar"""

lyrics = """
Liguei pra ouvir a tua voz \
Mas diz se não tiveres a sós \
Eu sei que tenho escutas, tenho meo tenho zon e tenho Vodafone \
Amigos coloridos, tenho vários benefícios nunca friend-zone \
Tipo esse burro do teu ex-damo com bué perfis \
'Tava na escola em frente a um quadro \
Da única vez que ele viu giz \
"""

# Format the message with the given style and lyrics
lyrics_message = prompt_template.format_messages(style=style, text=lyrics)

In [ ]:
lyrics_message

[SystemMessage(content='Translate the text that is delimited by triple backticks into a style that is English UK very polite and respectful as if you were royalty, if necessary corrects the grammar.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="text: ```\nLiguei pra ouvir a tua voz Mas diz se não tiveres a sós Eu sei que tenho escutas, tenho meo tenho zon e tenho Vodafone Amigos coloridos, tenho vários benefícios nunca friend-zone Tipo esse burro do teu ex-damo com bué perfis 'Tava na escola em frente a um quadro Da única vez que ele viu giz ```", additional_kwargs={}, response_metadata={})]

In [ ]:
# Call the LLM to translate to the style of the customer message
response = llm.invoke(lyrics_message)
response

AIMessage(content='I beseech thee to lend me thine ear, but do tell me if thou art not alone. I am aware that I am being listened to, for I possess Meo, Zon, and Vodafone. I have friends with benefits, never in the friend-zone. Like that fool of thy former lover with numerous profiles. He was at school in front of a blackboard, the only time he saw chalk.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 140, 'total_tokens': 226, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ac95e408-86b6-47dd-bad0-09f56b6aa308-0')

### Output Parsers

#### Motivation
Sometimes you want the LLM to output the answer in a given format.
Let's start with defining how we would like the LLM output to look like:

Example of a product review output

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

Example of customer review output

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

Example of prompt to extract the product information from the review

In [ ]:
# Define the system and human message templates
system_message_template = SystemMessagePromptTemplate.from_template(
    """For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? 
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product 
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, 
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value
"""
)

human_message_template = HumanMessagePromptTemplate.from_template(
    "text: {text}"
)

# Combine them into a chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    system_message_template,
    human_message_template,
])

In [ ]:
messages = prompt_template.format_messages(text=customer_review)

In [ ]:
# Call the LLM to translate to the style of the customer message
response = llm.invoke(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": [
        "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
    ]
}


In [ ]:
type(response.content)

str

#### Solution

In [ ]:
# Define the system and human message templates
system_message_template = SystemMessagePromptTemplate.from_template(
    """For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? 
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

{format_instructions}
"""
)

human_message_template = HumanMessagePromptTemplate.from_template(
    "text: {text}"
)

# Combine them into a chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    system_message_template,
    human_message_template,
])

##### Option 1: StructuredOutputParser and ResponseSchema

In [ ]:
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import ResponseSchema

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser1 = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions1 = output_parser1.get_format_instructions()

In [ ]:
print(format_instructions1)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [ ]:
chain1 = prompt_template | llm | output_parser1
output1 = chain1.invoke({"text": customer_review, "format_instructions": format_instructions1})

In [ ]:
print(output1)

{'gift': True, 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}


##### Option 2: PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class ProductReview(BaseModel):
    gift: bool = Field(
        description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown."
    )
    delivery_days: int = Field(
        description="How many days did it take for the product to arrive? If this information is not found, output -1."
    )
    price_value: List[str] = Field(
        description="Extract any sentences about the value or price, and output them as a comma-separated Python list."
    )


In [ ]:
from langchain.output_parsers import PydanticOutputParser

output_parser2 = PydanticOutputParser(pydantic_object=ProductReview)

In [ ]:
format_instructions2 = output_parser2.get_format_instructions()

In [ ]:
chain2 = prompt_template | llm | output_parser2
output2 = chain2.invoke({"text": customer_review, "format_instructions": format_instructions2})

In [ ]:
output2

ProductReview(gift=True, delivery_days=2, price_value=["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."])

In [ ]:
type(chain2)

langchain_core.runnables.base.RunnableSequence

### Memory

#### Motivation
When you interact with this LLM they typically don't remember what you say.
Which is useful in conversation for example.

In [ ]:
response1 = llm.invoke("Hello my name is Tiago.")
print(response1.content)

Hello Tiago, nice to meet you! How can I assist you today?


In [ ]:
response1 = llm.invoke("What is my name?")
print(response1.content)

I'm sorry, I do not have access to personal information such as your name.


#### Solution

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

Let's build a conversation chain

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')
prompt = ChatPromptTemplate(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

In [ ]:
# Clear the memory
memory.clear()

In [ ]:
conversation = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
    memory=memory,
)

In [ ]:
conversation_result = conversation.invoke("Hello my name is Tiago.")

In [ ]:
conversation_result['chat_history']

[HumanMessage(content='Hello my name is Tiago.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Tiago! How can I assist you today?', additional_kwargs={}, response_metadata={})]

In [ ]:
conversation_result = conversation.invoke("what is my name?")

In [ ]:
conversation_result['chat_history']

[HumanMessage(content='Hello my name is Tiago.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Tiago! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Tiago.', additional_kwargs={}, response_metadata={})]

In [ ]:
conversation_result['text']

'Your name is Tiago.'

### Chains

#### Motivation
When you want to be modular and reuse prompts you already have. It also simplifies the flow of information.

In [ ]:
answer = get_completion("""Give me 6 pairs of (product names | reviews) the reviews migth be in different languages,
                          but each product shoud only contain one review, each review should contain at least 30 words.\
                          The output should be in json format with two main keys 'products' and 'reviews' \
                           take into account it is suposse to save on a pandas dataframe. """)

In [ ]:
import pandas as pd
df = pd.read_json(answer)
df.rename(columns={'products': 'Product', 'reviews': 'Review'}, inplace=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_9856\3897400281.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(answer)


In [ ]:
df

,Product,Review
0,iPhone 12 Pro,The iPhone 12 Pro is an amazing phone with a s...
1,AirPods Pro,Les AirPods Pro sont incroyables! La qualité d...
2,Nintendo Switch,La Nintendo Switch es una consola increíble. L...
3,Dyson V11,El Dyson V11 es una aspiradora potente y efici...
4,Instant Pot,Instant Pot is a game-changer in the kitchen! ...
5,Fitbit Versa 3,O Fitbit Versa 3 é um ótimo smartwatch. A tela...


Imagine you want to answer customer reviews, and those reviews might be in different languages.

And from those reviews you want to extract a summary of the review.

Finally you want to answer the review in the same language.

#### Solution

#### SequentialChain

In [ ]:
from langchain.chains import SequentialChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [ ]:
# Define the language model
llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")

# Define the prompt template for translation
first_prompt = ChatPromptTemplate.from_messages(
    [   
        SystemMessagePromptTemplate.from_template("Translate the following review to English."),
        HumanMessagePromptTemplate.from_template("{Review}"),
    ]
)

# Chain for translating the review to English
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="english_Review")

In [ ]:
# Define the prompt template for summarization
second_prompt = ChatPromptTemplate.from_messages(
    [   
        SystemMessagePromptTemplate.from_template("Summarize the following review in 1 sentence."),
        HumanMessagePromptTemplate.from_template("{english_Review}")
    ]
)

# Chain for summarizing the English review
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


In [ ]:
# Define the prompt template to identify the language of the review
third_prompt = ChatPromptTemplate.from_messages(
    [   
        SystemMessagePromptTemplate.from_template("Identify the language of the following review."),
        HumanMessagePromptTemplate.from_template("{Review}")
    ]
)

# Chain for identifying the language of the review
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [ ]:
# Define the prompt template for a follow-up message
fourth_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("Write a follow-up response to the following summary in the specified language."),
        HumanMessagePromptTemplate.from_template("Summary: {summary}\n\nLanguage: {language}")
    ]
)

# Chain for generating a follow-up message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [ ]:
# Overall chain: input = Review, and output = English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["english_Review", "summary", "followup_message"],
    verbose=True
)


In [ ]:
review = df.Review[1]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Les AirPods Pro sont incroyables! La qualité du son est incroyable et la réduction du bruit fonctionne très bien. Ils sont confortables à porter et la durée de vie de la batterie est excellente.',
 'english_Review': 'The AirPods Pro are amazing! The sound quality is incredible and the noise cancellation works very well. They are comfortable to wear and the battery life is excellent.',
 'summary': 'The reviewer is highly impressed with the AirPods Pro, praising their sound quality, noise cancellation, comfort, and battery life.',
 'followup_message': "Merci pour votre commentaire élogieux sur les AirPods Pro ! Nous sommes ravis que vous ayez apprécié la qualité sonore, la suppression du bruit, le confort et l'autonomie de la batterie de ces écouteurs. Vos éloges nous encouragent à continuer à proposer des produits de haute qualité. N'hésitez pas à nous faire part de vos expériences ou besoins supplémentaires à l'avenir."}

#### RouterChain

Imagine we want to have multiple prompts and we want to choose one based on the output of the previous prompt.

In [1]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{text}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{text}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{text}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{text}"""


default_template = """You are a helpful AI assistant. \
You can answer questions on a wide range of topics. \
If you're not sure about an answer, you can say so.

Here is the question:
{text}"""

Now from those prompts lets create prompt tempates and chains.

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

templates = [physics_template, math_template, history_template, computerscience_template, default_template]
names = ["Physics", "Math", "History", "Computer Science", "Default"]
destination_chains = {}

human_message_template = HumanMessagePromptTemplate.from_template(
    "text: {text}"
)

for system_template, name in zip(templates, names):
    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_template),
        human_message_template,
    ])

    destination_chains[name] = LLMChain(llm=llm, prompt=prompt_template)

C:\Users\tiago\AppData\Local\Temp\ipykernel_22076\4125052752.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  destination_chains[name] = LLMChain(llm=llm, prompt=prompt_template)


In [6]:
destinations = list(destination_chains.keys()) 
destinations_str = "["+", ".join(destinations)+"]"
destinations_str

'[Physics, Math, History, Computer Science, Default]'

Let's create a Pydantic Class to extract the destination chain.

In [85]:
from typing import Literal
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class RouteQuery(BaseModel):
    """Given a raw text input to a language model select the destination that best suits the input."""
    destination: Literal["Physics", "Math", "History", "Computer Science", "Default"] 

output_parser = PydanticOutputParser(pydantic_object=RouteQuery)

Now we need to create a main chain that will be used in the router chain.

In [74]:
# Define the system and human message templates
system_message_template = SystemMessagePromptTemplate.from_template(
    """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \

Available prompts: {destinations_str}

If any of the prompts are not suitable for the input, output 'Default'.

{format_instructions}
"""
)

human_message_template = HumanMessagePromptTemplate.from_template(
    "text: {text}"
)

# Combine them into a chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    system_message_template,
    human_message_template,
])

In [ ]:
text = "What is 1+1?"

# Define the chain that will be used to route the input text to the correct destination
route_chain = prompt_template | llm | output_parser

# Extract the Pydantic object from the output of the route chain
route_object = route_chain.invoke({"text": text, "destinations_str":destinations_str, "format_instructions": output_parser.get_format_instructions()})

# Extract the destination from the Pydantic object
destination = route_object.destination

# Use the destination to invoke the correct destination chain
answer = destination_chains[destination].invoke({"text": text})

### Final ChatBot

In [122]:
# Product Database
PRODUCT_DATABASE = {
    "Computers and Laptops": [
        "TechPro Ultrabook",
        "BlueWave Gaming Laptop",
        "PowerLite Convertible",
        "TechPro Desktop",
        "BlueWave Chromebook"
    ],
    "Smartphones and Accessories": [
        "SmartX ProPhone",
        "MobiTech PowerCase",
        "SmartX MiniPhone",
        "MobiTech Wireless Charger",
        "SmartX EarBuds"
    ],
    "Televisions and Home Theater Systems": [
        "CineView 4K TV",
        "SoundMax Home Theater",
        "CineView 8K TV",
        "SoundMax Soundbar",
        "CineView OLED TV"
    ],
    "Gaming Consoles and Accessories": [
        "GameSphere X",
        "ProGamer Controller",
        "GameSphere Y",
        "ProGamer Racing Wheel",
        "GameSphere VR Headset"
    ],
    "Audio Equipment": [
        "AudioPhonic Noise-Canceling Headphones",
        "WaveSound Bluetooth Speaker",
        "AudioPhonic True Wireless Earbuds",
        "WaveSound Soundbar",
        "AudioPhonic Turntable"
    ],
    "Cameras and Camcorders": [
        "FotoSnap DSLR Camera",
        "ActionCam 4K",
        "FotoSnap Mirrorless Camera",
        "ZoomMaster Camcorder",
        "FotoSnap Instant Camera"
    ]
}

def format_product_database():
    """Format the product database for prompt templates"""
    categories = "\n".join(f"- {category}" for category in PRODUCT_DATABASE.keys())
    products = "\n".join(
        f"{category}:\n" + "\n".join(f"  - {product}" for product in products)
        for category, products in PRODUCT_DATABASE.items()
    )
    return categories, products

In [ ]:
import pickle

# Load the dictionary from the pickle file
with open('products_catalog.pkl', 'rb') as handle:
  products_catalog = pickle.load(handle)

In [196]:
categories, products = format_product_database()

In [247]:
from typing import List, Optional
import json

# Initialize the LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [248]:
# Define Pydantic models for structured output
class ProductCategory(BaseModel):
    category: Optional[str] = Field(None, description="The product category")
    products: Optional[List[str]] = Field(None, description="List of products mentioned")

class ProductQueryResult(BaseModel):
    results: List[ProductCategory]

product_parser = PydanticOutputParser(pydantic_object=ProductQueryResult)

In [249]:
# Product identification prompt templates
PRODUCT_SYSTEM_TEMPLATE = """
You are a product identification system for an electronics store.
Your task is to analyze customer service queries and identify mentioned products and categories.

Available categories:
{categories}

Available products:
{products}

{format_instructions}

Ensure your response follows the exact format specified in the instructions.
"""

PRODUCT_HUMAN_TEMPLATE = """
Customer Query: {customer_input}
"""

product_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(PRODUCT_SYSTEM_TEMPLATE),
    HumanMessagePromptTemplate.from_template(PRODUCT_HUMAN_TEMPLATE)
])

In [250]:
# Product identification chain
product_chain = LLMChain(
    llm=llm,
    output_parser=product_parser,
    prompt=product_prompt,
    verbose=False
)

In [251]:
result = product_chain.invoke({"customer_input":"What kind of laptops and CineView 4K TV do you have?", "categories": categories, "products": products, "format_instructions": product_parser.get_format_instructions()})

In [252]:
result

{'customer_input': 'What kind of laptops and CineView 4K TV do you have?',
 'categories': '- Computers and Laptops\n- Smartphones and Accessories\n- Televisions and Home Theater Systems\n- Gaming Consoles and Accessories\n- Audio Equipment\n- Cameras and Camcorders',
 'products': 'Computers and Laptops:\n  - TechPro Ultrabook\n  - BlueWave Gaming Laptop\n  - PowerLite Convertible\n  - TechPro Desktop\n  - BlueWave Chromebook\nSmartphones and Accessories:\n  - SmartX ProPhone\n  - MobiTech PowerCase\n  - SmartX MiniPhone\n  - MobiTech Wireless Charger\n  - SmartX EarBuds\nTelevisions and Home Theater Systems:\n  - CineView 4K TV\n  - SoundMax Home Theater\n  - CineView 8K TV\n  - SoundMax Soundbar\n  - CineView OLED TV\nGaming Consoles and Accessories:\n  - GameSphere X\n  - ProGamer Controller\n  - GameSphere Y\n  - ProGamer Racing Wheel\n  - GameSphere VR Headset\nAudio Equipment:\n  - AudioPhonic Noise-Canceling Headphones\n  - WaveSound Bluetooth Speaker\n  - AudioPhonic True Wirele

In [254]:
def get_product_by_name(name):
    return products_catalog.get(name, None)

def get_products_by_category(category):
    return [product for product in products_catalog.values() if product["category"] == category]

def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            # Check if the data is a instance of ProductCategory
            if isinstance(data, ProductCategory):

                # Check if the category is specified
                if data.category:
                    #print(f"Category: {data.category}")
                    category_products = get_products_by_category(data.category)
                    for product in category_products:
                        output_string += json.dumps(product, indent=4) + "\n"
                
                # Check if the products are specified
                if data.products:
                    for product_name in data.products:
                        #print(f"Product: {product_name}")
                        product = get_product_by_name(product_name)
                        if product:
                            output_string += json.dumps(product, indent=4) + "\n"
                        else:
                            print(f"Error: Product '{product_name}' not found")
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string

In [255]:
product_info = generate_output_string(result["text"].results)

In [256]:
# Customer service prompt templates
SERVICE_SYSTEM_TEMPLATE = """
You are a friendly and helpful customer service assistant for a large electronics store.
Follow these guidelines:
1. Provide concise, helpful responses
2. Ask relevant follow-up questions when needed
3. Show understanding of specific products mentioned
4. Be professional but conversational in tone
5. Focus on solving the customer's immediate needs
"""

SERVICE_HUMAN_TEMPLATE = """
Product Information from Query:
{product_info}

Customer Query: {customer_input}
"""

service_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(SERVICE_SYSTEM_TEMPLATE),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template(SERVICE_HUMAN_TEMPLATE)
])

In [258]:
memory = ConversationBufferMemory(
        memory_key="chat_history",
        input_key="customer_input",
        return_messages=True
    )

In [259]:
# Product identification chain
service_chain = LLMChain(
    llm=llm,
    prompt=service_prompt,
    output_key="response",
    memory=memory,
    verbose=False
)

In [260]:
service_chain.invoke({"customer_input":"What kind of laptops and CineView 4K TV do you have?", "product_info": product_info, "chat_history": []})

{'customer_input': 'What kind of laptops and CineView 4K TV do you have?',
 'product_info': '{\n    "name": "TechPro Ultrabook",\n    "category": "Computers and Laptops",\n    "brand": "TechPro",\n    "model_number": "TP-UB100",\n    "warranty": "1 year",\n    "rating": 4.5,\n    "features": [\n        "13.3-inch display",\n        "8GB RAM",\n        "256GB SSD",\n        "Intel Core i5 processor"\n    ],\n    "description": "A sleek and lightweight ultrabook for everyday use.",\n    "price": 799.99\n}\n{\n    "name": "BlueWave Gaming Laptop",\n    "category": "Computers and Laptops",\n    "brand": "BlueWave",\n    "model_number": "BW-GL200",\n    "warranty": "2 years",\n    "rating": 4.7,\n    "features": [\n        "15.6-inch display",\n        "16GB RAM",\n        "512GB SSD",\n        "NVIDIA GeForce RTX 3060"\n    ],\n    "description": "A high-performance gaming laptop for an immersive experience.",\n    "price": 1199.99\n}\n{\n    "name": "PowerLite Convertible",\n    "category

In [ ]:
class CustomerServiceBot:
    def __init__(self):
        # Initialize memories
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            input_key="customer_input",
            return_messages=True
        )
        
        # Product identification chain
        self.product_chain = LLMChain(
            llm=llm,
            output_parser=product_parser,
            prompt=product_prompt,
            verbose=False
        )

        # Customer service chain
        self.service_chain = LLMChain(
            llm=llm,
            prompt=service_prompt,
            output_key="response",
            memory=self.memory,
            verbose=False
        )

        self.categories, self.products = format_product_database()
        self.product_format_instructions = product_parser.get_format_instructions()


    def final_chain(self, customer_input):

        # Identify products and categories from the customer input
        result = self.product_chain.invoke({"customer_input": customer_input, 
                                   "categories": self.categories, 
                                   "products": self.products, 
                                   "format_instructions": self.product_format_instructions})

        # Generate product information output string
        self.product_info = generate_output_string(result["text"].results)

        # Process the customer service query
        final_result = self.service_chain.invoke({"customer_input": customer_input,
                                    "product_info": self.product_info,
                                    "chat_history": []})
                                   
        return final_result
    
    def process_message(self, user_input: str) -> str:
        """Process a user message and generate a response"""
        try:
            final_result = self.final_chain(user_input)
            return final_result["response"]

        except Exception as e:
            print(f"Error processing message: {e}")
            return "I apologize, but I encountered an error processing your request. Could you please rephrase your question?"

In [ ]:
bot = CustomerServiceBot()
print("Customer Service Bot initialized. Type 'exit' or 'quit' to end the conversation.")

while True:
    user_input = input("You: ").strip()
    
    if user_input.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
        
    try:
        response = bot.process_message(user_input)
        print(f"Bot: {response}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please try again with a different query.")

Customer Service Bot initialized. Type 'exit' or 'quit' to end the conversation.


Bot: Hello! How can I assist you with your electronics needs today?
Bot: We have a variety of laptops to suit different needs and budgets. Here are a few options:

1. **TechPro Ultrabook**:
   - **Price**: $799.99
   - **Features**: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   - **Description**: A sleek and lightweight ultrabook for everyday use.
   - **Warranty**: 1 year
   - **Rating**: 4.5

2. **BlueWave Gaming Laptop**:
   - **Price**: $1199.99
   - **Features**: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   - **Description**: A high-performance gaming laptop for an immersive experience.
   - **Warranty**: 2 years
   - **Rating**: 4.7

3. **PowerLite Convertible**:
   - **Price**: $699.99
   - **Features**: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   - **Description**: A versatile convertible laptop with a responsive touchscreen.
   - **Warranty**: 1 year
   - **Rating**: 4.3

4. **BlueWave Chromebook**:
   - **Price**: $24

In [271]:
bot.memory.chat_memory.messages

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you with your electronics needs today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What can you tell me about laptops?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='We have a variety of laptops to suit different needs and budgets. Here are a few options:\n\n1. **TechPro Ultrabook**:\n   - **Price**: $799.99\n   - **Features**: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor\n   - **Description**: A sleek and lightweight ultrabook for everyday use.\n   - **Warranty**: 1 year\n   - **Rating**: 4.5\n\n2. **BlueWave Gaming Laptop**:\n   - **Price**: $1199.99\n   - **Features**: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060\n   - **Description**: A high-performance gaming laptop for an immersive experience.\n   - **Warranty**: 2 years\n   - **Rating**: 4.7\n\n3. **PowerLite Convertible**:\n   -